In [1]:
from aocd import get_data
from itertools import combinations
from collections import deque
raw_data = get_data(day=9, year=2025)
data = [[int(item) for item in line.split(",")] for line in raw_data.splitlines()]

## Part 1

In [2]:
max_area = 0
for point1, point2 in combinations(data, 2):
    area = (abs((point1[0] - point2[0])) + 1) * (abs(point1[1] - point2[1]) + 1)
    max_area = max(area, max_area) # No need for an if statement
print(max_area)

4748769124


## Part 2

In [ ]:
# Ray-casting approach

red_tiles = set(tuple(row) for row in data)
green_tiles = set()

# Store boundary segments for the fast validity check
# Vertical segments: [x, min_y, max_y]
# Horizontal segments: [y, min_x, max_x]
vertical_segments = []
horizontal_segments = []

# Iterate through consecutive red tile pairs (wrapping last to first)
# to find the green tiles connecting them
for (x1, y1), (x2, y2) in zip(data, data[1:] + [data[0]]):
    if x1 == x2:
        vertical_segments.append([x1, min(y1, y2), max(y1, y2)])
        for y in range(min(y1, y2) + 1, max(y1, y2)):
            green_tiles.add((x1, y))
    elif y1 == y2:
        horizontal_segments.append([y1, min(x1, x2), max(x1, x2)])
        for x in range(min(x1, x2) + 1, max(x1, x2)):
            green_tiles.add((x, y1))

boundary_tiles = red_tiles.union(green_tiles)

def is_inside(px, py):
    count = 0
    for segment in vertical_segments:
        if segment[0] > px and segment[1] <= py < segment[2]: # Include one endpoint and exclude the other to count cases where the ray passes exactly through the vertex where two segments meet
            count += 1
    is_odd = count % 2 !=0
    return is_odd 

# ORIGINAL SLOW IMPLEMENTATION - DO NOT USE
# This checks every single tile in the rectangle, which is O(width * height).
# For rectangles spanning coordinates ~80,000-98,000, this could mean checking

# def is_valid_rectangle(x1, y1, x2, y2):
#     for x in range(min(x1, x2), max(x1, x2) + 1):
#         for y in range(min(y1, y2), max(y1, y2) + 1):
#             if not((x, y) in boundary_tiles or is_inside(x, y)):
#                 return False
#     return True

def is_in_boundary_or_inside(x, y):
    """Helper: check if a single point is valid (on boundary or inside polygon)."""
    return ((x, y) in boundary_tiles or is_inside(x, y))

def is_valid_rectangle(x1, y1, x2, y2):
    """
    FAST IMPLEMENTATION - O(number of segments) instead of O(width * height).
    
    A rectangle is valid if:
    1. All 4 corners are valid (on boundary or inside polygon)
    2. No boundary segment passes through the interior (which would mean
       part of the rectangle is inside the polygon and part is outside)
    """
    if not (is_in_boundary_or_inside(x1, y1) and is_in_boundary_or_inside(x2, y1) and is_in_boundary_or_inside(x1, y2) and is_in_boundary_or_inside(x2, y2)):
        return False
    
    # Check no vertical segment crosses through the rectangle's interior
    # A segment crosses if its x is strictly inside the rectangle's x-range
    # AND its y-range overlaps with the rectangle's y-range
    for segment in vertical_segments:
        if min(x1, x2) < segment[0] < max(x1, x2) and segment[2] > min(y1, y2) and segment[1] < max(y1, y2):
            return False
    # Same check
    for segment in horizontal_segments:
        if min(y1, y2) < segment[0] < max(y1, y2) and segment[2] > min(x1, x2) and segment[1] < max(x1, x2):
            return False
    return True
    
# Sort all pairs of red tiles by area (largest first)
# This way, the first valid rectangle we find is guaranteed to be the largest
sorted_combinations = sorted(combinations(data, 2), key=lambda pair: (abs(pair[0][0]-pair[1][0])+1) * (abs(pair[0][1]-pair[1][1])+1), reverse=True)
for point1, point2 in sorted_combinations:
    if is_valid_rectangle(point1[0], point1[1], point2[0], point2[1]):
        max_area = (abs(point1[0] - point2[0]) + 1) * (abs(point1[1] - point2[1]) + 1)
        break
        
print(max_area)

1525991432
